# Datasets Tutorial
Couple of scripts to generate dataset for ORB-SLAM3.

In [ ]:
%pip install torchdata

In [1]:
import cv2
import numpy as np
import tempfile
import shutil
from tqdm.notebook import tqdm
from pathlib import Path

In [5]:
# Maximum time difference that considers stere images are captured at same time
SYNC_DT_MAX_MSEC = 100 # msec

# Convert stereo camera images captured in Unity scene.
ds_dir = Path("datasets/sim_rotate_360_15deg_orig")
cam0 = ds_dir / "cam0" 
cam1 = ds_dir / "cam1"

# Output directory
out_dir = Path("datasets/sim_rotate_360_15deg")
cam0_out = out_dir / "cam0" / "data"
cam1_out = out_dir / "cam1" / "data"
imu0_out = out_dir / "imu0"
if out_dir.exists():
    shutil.rmtree(str(out_dir))
cam0_out.mkdir(parents=True)
cam1_out.mkdir(parents=True)
imu0_out.mkdir(parents=True)

cam0_ts = np.sort(np.array([int(x.name[0:-4]) for x in cam0.glob("*.png")]), axis=None)
cam1_ts = np.sort(np.array([int(x.name[0:-4]) for x in cam1.glob("*.png")]), axis=None)

# Synchronize stereo frames
timestamps = list()
for t in tqdm(cam0_ts):
    im = cam0 / ("%d.png" % t)
    tl = int(im.name[0:-4]) # nsec
    tr = cam1_ts[np.abs(cam1_ts - tl).argmin()] # nsec
    if np.abs(tl - tr) < SYNC_DT_MAX_MSEC * 1e6:
        shutil.copyfile(im, cam0_out / im.name)        
        shutil.copyfile(cam1 / ("%d.png" % tr), cam1_out / ("%d.png" % tl))
        timestamps.append(tl)
    else:
        print('Timestamp difference is too large: %f' % (np.abs(tl - tr) / 1e9))

# Save imu0.csv if exists
imu0_csv = ds_dir / "imu0" / "data.csv"
if imu0_csv.exists():
    shutil.copyfile(imu0_csv, imu0_out / "data.csv")

# Save timestamp_cam.txt
with open(out_dir / "timestamp_cam.txt", "w") as f:
    for t in timestamps:
        f.write("%d\n" % t)

  0%|          | 0/294 [00:00<?, ?it/s]

## Data Pipe

In [14]:
import re
import os

from operator import itemgetter
from torchdata.datapipes.iter import IterableWrapper

def VIDataPipe(uri):    
    """Vision Inertial Data Pipeline that fetches stereo intertial VSLAM dataset from Google Cloud Storage or local file system"""    
    
    CAM0_IMG_PAT = re.compile(r"cam0/data/[0-9]+[.]png")
    CAM1_IMG_PAT = re.compile(r"cam1/data/[0-9]+[.]png")
    IMU0_CSV_PAT = re.compile(r"imu0/data[.]csv")

    def collate_cam(data):
        path, im = data
        t = int(os.path.basename(path)[0:-4])
        im = cv2.imdecode(np.frombuffer(im.read(), np.uint8), cv2.IMREAD_GRAYSCALE)
        return t, im

    def classifier(data):
        path, _ = data
        if bool(IMU0_CSV_PAT.search(path)):
            return 0
        if bool(CAM0_IMG_PAT.search(path)):
            return 1
        elif bool(CAM1_IMG_PAT.search(path)):
            return 2
        else:
            return None

    class Compounder:
        def __init__(self, imu_dp):
            self._imu = np.asarray(list(imu_dp))
            self._t_last = 0

        def __call__(self, cam0, cam1):
            t = cam0[0]
            imu = self._imu[self._imu[:,0] > self._t_last]
            imu = imu[imu[:,0] <= t]
            self._t_last = t
            return t, cam0[1], cam1[1], imu
    
    # Demux stereo inertial dataset into three data pipes: Imu0, Cam0, Cam1
    if uri.startswith("gs://"):
        imu_dp, cam0_dp, cam1_dp = IterableWrapper([uri]) \
            .open_files_by_fsspec(mode="rb") \
            .load_from_bz2() \
            .load_from_tar() \
            .demux(3, classifier_fn=classifier, drop_none=True)
    elif uri.endswith(".tar.bz") or uri.endwith(".tbz"):
        imu_dp, cam0_dp, cam1_dp = IterableWrapper([uri]) \
            .open_files(mode="rb") \
            .load_from_bz2() \
            .load_from_tar() \
            .demux(3, classifier_fn=classifier, drop_none=True)        

    imu_dp = imu_dp \
        .parse_csv(skip_lines=1) \
        .map(lambda x: np.asarray(x).astype(np.double))
    
    # Sort by timestamp in asc order
    cam0_dp = cam0_dp.map(collate_cam)
    cam0_dp = IterableWrapper(sorted(list(cam0_dp), key=itemgetter(0)))    
    cam1_dp = cam1_dp.map(collate_cam)                 
        
    # Collate final data pipe
    dp = cam0_dp.zip_with_iter(cam1_dp, 
                               key_fn=itemgetter(0),
                               ref_key_fn=itemgetter(0),
                               merge_fn=Compounder(imu_dp),
                               keep_key=False)
    
    return dp

In [15]:
for t, _, _, _ in VIDataPipe('datasets/sim_rotate_360_15deg.tar.bz'):
    print(t)

1676077410017000000
1676077410171000000
1676077410341000000
1676077410443000000
1676077410593000000
1676077410717000000
1676077410885000000
1676077411054000000
1676077411213000000
1676077411364000000
1676077411514000000
1676077411659000000
1676077411779000000
1676077411889000000
1676077412110000000
1676077412440000000
1676077412610000000
1676077412789000000
1676077412952000000
1676077413103000000
1676077413215000000
1676077413411000000
1676077413548000000
1676077413720000000
1676077413919000000
1676077414133000000
1676077414294000000
1676077414464000000
1676077414566000000
1676077414667000000
1676077414801000000
1676077414981000000
1676077415123000000
1676077415311000000
1676077415445000000
1676077415602000000
1676077415739000000
1676077415906000000
1676077416146000000
1676077416347000000
1676077416530000000
1676077416681000000
1676077416864000000
1676077416973000000
1676077417134000000
1676077417270000000
1676077417428000000
1676077417595000000
1676077417687000000
1676077417823000000
